In [43]:
import numpy as np
import pandas as pd
import warnings

In [44]:
pd.set_option('display.max_columns', 500)
warnings.filterwarnings('ignore')
%matplotlib inline

In [45]:
# Load dataset: listing 
listing_df = pd.read_csv('C:\\DataScience\\00_AnalysisProjects\\AirbnbAnalysis\\Data\\London_listings.csv')
listing_df.shape   # (77096, 96)

(77096, 96)

In [46]:
# Load dataset: calendar
calendar_df = pd.read_csv('C:\\DataScience\\00_AnalysisProjects\\AirbnbAnalysis\\Data\\London_calendar.csv')
calendar_df.shape 

(28139675, 4)

In [47]:
# View data structure and content
calendar_df.head()

,listing_id,date,available,price
0,9554,2019-11-25,t,$35.00
1,9554,2019-11-24,t,$35.00
2,9554,2019-11-23,t,$39.00
3,9554,2019-11-22,t,$39.00
4,9554,2019-11-21,t,$35.00


In [48]:
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28139675 entries, 0 to 28139674
Data columns (total 4 columns):
listing_id    int64
date          object
available     object
price         object
dtypes: int64(1), object(3)
memory usage: 858.8+ MB


In [50]:
# Check missing data
def Check_Missing_Data(df):    
    # count all missing values of each column
    total = df.isnull().sum().sort_values(ascending=True)
    # calculate percentage of null values for each column
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=True)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

Check_Missing_Data(calendar_df)

,Total,Percent
listing_id,0,0.000000
date,0,0.000000
available,0,0.000000
price,18671601,0.663533


In [51]:
# Convert price from string to float
calendar_df['price'] = (calendar_df['price'].str.replace(r'[^-+\d.]', '').astype(float))

# Remove all rows with missing data
calendar_df = calendar_df[calendar_df.price > 0]
Check_Missing_Data(calendar_df)

,Total,Percent
listing_id,0,0.0
date,0,0.0
available,0,0.0
price,0,0.0


In [17]:
# Recheck dimension of calendar after removing missing price
calendar_df.shape 

(9468074, 4)

In [52]:
# calculate sum and average of price by date
df1  = calendar_df.groupby("date")[["price"]].sum()
df1["mean"]  = calendar_df.groupby("date")[["price"]].mean()
df1.columns = ["Total", "Average"]
df1.head()

,Total,Average
date,,
2018-12-07,962389.0,201.210328
2018-12-08,1460167.0,159.076915
2018-12-09,2304126.0,133.117222
2018-12-10,2660298.0,132.868744
2018-12-11,2726906.0,131.753684


In [53]:
# set column date as index
df2 = calendar_df.set_index("date")
# convert string to datetime
df2.index = pd.to_datetime(df2.index)
# resample data by month
df2 =  df2[["price"]].resample("M").mean()
df2.head()

,price
date,
2018-12-31,139.711064
2019-01-31,126.043959
2019-02-28,123.875743
2019-03-31,132.424334
2019-04-30,139.654084


### Visualization

In [54]:
import plotly as py
from plotly.offline import iplot, plot, init_notebook_mode, download_plotlyjs
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import plotly.offline as offline

In [55]:
trace1 = go.Scatter(
    x = df1.index,
    y = df1["Total"]
)
data = [trace1]
layout = go.Layout(
    title = "Price by each time",
    xaxis  = dict(title = "Time"),
    yaxis = dict(title = "Total ($)")
)
trace2 = go.Scatter(
    x = df1.index,
    y = df1["Average"]
)

data2 = [trace2]
layout2 = go.Layout(
    title = "Price by each time",
    xaxis  = dict(title = "Time"),
    yaxis = dict(title = "Mean ($)")
)
fig = go.Figure(data = data, layout = layout)
fig2 = go.Figure(data = data2, layout = layout2)


In [56]:
# To view interactive plots, click link below : 
# https://nbviewer.jupyter.org/github/ntmai03/Airbnb/blob/master/AirbnbTimeSeriesAnalysis.ipynb
offline.iplot(fig)

In [28]:
# To view interactive plots, click link below : 
# https://nbviewer.jupyter.org/github/ntmai03/Airbnb/blob/master/AirbnbTimeSeriesAnalysis.ipynb
offline.iplot(fig2)

In [9]:
trace3 = go.Scatter(
    x = df2.index[:-1],
    y = df2.price[:-1]
)
layout3 = go.Layout(
    title = "Average price by month",
    xaxis = dict(title = "time"),
    yaxis = dict(title = "Price")
)
data3 = [trace3]
fig3 = go.Figure(data= data3, layout= layout3)

# To view interactive plots, click link below : 
# https://nbviewer.jupyter.org/github/ntmai03/Airbnb/blob/master/AirbnbTimeSeriesAnalysis.ipynb
offline.iplot(fig3)

### Time series stationarity and using statistic test

In [31]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [32]:
def draw_interactive_graph(mode):
    df1.index = pd.to_datetime(df1.index)
    decomposition = seasonal_decompose(df1[[mode]])
    trace4_1 = go.Scatter(
        x = decomposition.observed.index, 
        y = decomposition.observed[mode],
        name = "Observed"
    )
    trace4_2 = go.Scatter(
        x = decomposition.trend.index,
        y = decomposition.trend[mode],
        name = "Trend"
    )
    trace4_3 = go.Scatter(
        x = decomposition.seasonal.index,
        y = decomposition.seasonal[mode],
        name = "Seasonal"
    )
    trace4_4 = go.Scatter(
        x = decomposition.resid.index,
        y = decomposition.resid[mode],
        name = "Resid"
    )

    fig = py.tools.make_subplots(rows=4, cols=1, subplot_titles=('Observed', 'Trend',
                                                              'Seasonal', 'Residiual'))
    # append trace into fig
    fig.append_trace(trace4_1, 1, 1)
    fig.append_trace(trace4_2, 2, 1)
    fig.append_trace(trace4_3, 3, 1)
    fig.append_trace(trace4_4, 4, 1)

    fig['layout'].update( title='Descompose with TimeSeries')
    offline.iplot(fig)

In [33]:
draw_interactive_graph("Average")

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



In [34]:
draw_interactive_graph("Total")

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



In [38]:
calendar_clean = calendar_df.dropna()
calendar_clean.set_index("date", inplace = True)
calendar_clean.head()

,listing_id,available,price
date,,,
2019-11-25,9554,t,35.0
2019-11-24,9554,t,35.0
2019-11-23,9554,t,39.0
2019-11-22,9554,t,39.0
2019-11-21,9554,t,35.0


In [39]:
calendar_clean.index = pd.to_datetime(calendar_clean.index)
number_hire_room = calendar_clean.resample("M")[["price"]].count()
total_price_each_month  = calendar_clean.resample("M")[["price"]].sum()

In [40]:
trace5 = go.Scatter(
    x = number_hire_room.index[:-1],
    y = number_hire_room.price[:-1]
)
data5 = [trace5]
layout5 = go.Layout(
    title = "Number of Hire Room by Month in London",
    xaxis = dict(title = "Month"),
    yaxis = dict(title = "Number hirde")
)
fig5  = go.Figure(data = data5, layout = layout5)

offline.iplot(fig5)

In [41]:
trace6 = go.Scatter(
    x = number_hire_room.index[:-1],
    y = number_hire_room.price[:-1]/number_hire_room.price[0]
)
data6 = [trace6]
layout6 = go.Layout(
    title = "the ratio of the number of rooms compare with the first month",
    xaxis = dict(title = "Month"),
    yaxis = dict(title = "Ratio")
)
fig6 = go.Figure(data = data6, layout = layout6)

offline.iplot(fig6)

In [25]:
from scipy import stats

In [42]:
a = calendar_clean.index.month
# calendar_clean["Month"] = a
calendar_clean = calendar_clean.assign(Month = a)
calendar_clean.head()

,listing_id,available,price,Month
date,,,,
2019-11-25,9554,t,35.0,11
2019-11-24,9554,t,35.0,11
2019-11-23,9554,t,39.0,11
2019-11-22,9554,t,39.0,11
2019-11-21,9554,t,35.0,11


In [27]:


result = []
for i in range(1,13):
    result.append(np.array([calendar_clean[calendar_clean["Month"] == i].price]))



In [28]:
data_score = []
for i in range(11):
    score = stats.ttest_rel(result[i][0][:64911],result[-1][0][:64911])
    data_score.append((score[0], score[1]))

In [29]:
score_board = pd.DataFrame(data = data_score, columns = ["Test Statistic", "P_value"])
score_board["Month"] = range(1, 12)
score_board.set_index("Month", inplace = True)
score_board

,Test Statistic,P_value
Month,,
1,-14.907433,3.571048e-50
2,-15.748268,8.957391e-56
3,-14.036928,1.077138e-44
4,-10.806497,3.382590e-27
5,-11.480491,1.769421e-30
6,-6.952102,3.632667e-12
7,-2.532751,1.131948e-02
8,-0.698523,4.848527e-01
9,-5.631242,1.796535e-08


In [30]:
offline.iplot(fig3)